# Property Search Playground

Perform Auto-merging retrieval using index only on owner name and property legal name.  
We will use FAISS index.


## Setting and Preparations

In [1]:
import pandas as pd
from llama_index.core import Document, ServiceContext
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, get_response_synthesizer
from llama_index.llms.openai import OpenAI
import os
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo
from llama_index.vector_stores.faiss import FaissVectorStore

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import ipywidgets as widgets
#widgets.IntSlider()

In [2]:
variables_to_define = [
    "OPENAI_API_KEY",
    "embeddings_llm",
    "embeddings_cache_folder",
    "data_path",
    ]
def load_env_file(file_path):
    try:
        with open(file_path, 'r') as file:
            for line in file:
                # Remove whitespace and comments
                line = line.strip()
                if line and not line.startswith('#'):
                    # Split by the first '=' character
                    key, value = line.split('=', 1)
                    # Remove any surrounding quotes from the value
                    value = value.strip().strip('"').strip("'")
                    # Set the environment variable
                    if key in variables_to_define:
                        os.environ[key.strip()] = value
    except FileNotFoundError:
        print(f"Error: {file_path} not found.")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

# Load environment variables from .env file
project_path = "/usr/local/stage3technical/var/sand/property-rag-search"
load_env_file(os.path.join(project_path,'.env'))

# Define variables from environment variables
embeddings_llm = os.getenv("embeddings_llm")
embeddings_cache_folder = os.getenv("embeddings_cache_folder")
openai_api_key = os.getenv("OPENAI_API_KEY")
data_path = os.getenv("data_path")

### Define models

In [3]:
# Embedding model
embedding_model = HuggingFaceEmbedding(model_name=embeddings_llm,
                                      cache_folder=embeddings_cache_folder,
                                      embed_batch_size=32,
                                      )

# Generation model
generation_llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key=openai_api_key,  # uses OPENAI_API_KEY env var by default
)

In [5]:
from llama_index.core.llms import ChatMessage
messages = [
    ChatMessage(
        role="system", content="You are a helpful assistant that can answer questions about the world."
    ),
    ChatMessage(role="user", content="What is your LLM model name?"),
]
resp = generation_llm.chat(messages)
print(resp)

assistant: I am based on OpenAI's GPT-3 model. However, I don't have a specific model name beyond that. If you have any questions or need assistance, feel free to ask!


In [6]:
# Mapping columns for preprocessing
key_replacements = {
    "propYear": "property year",
    "propID": "property ID",
    "geoID": "geographical ID",
    "propType": "property type",
    "propSubType": "property subtype",
    "propCategoryCode": "property category code",
    "legalDescription": "legal description",
    "dbaName": "doing business as name",
    "propCreateDate": "property creation date",
    "situsBldgNum": "situs building number",
    "situsStreetPrefix": "situs street prefix",
    "situsStreetName": "situs street name",
    "situsStreetSuffix": "situs street suffix",
    "situsUnit": "situs unit",
    "situsCity": "situs city",
    "situsZip": "situs ZIP",
    "situsConcat": "situs concatenated",
    "situsConcatShort": "situs concatenated short",
    "ownerID": "owner ID",
    "ownerName": "owner name",
    "ownerNameAddtl": "owner name additional",
    "ownerAddrLine1": "owner address line 1",
    "ownerAddrLine2": "owner address line 2",
    "ownerAddrCity": "owner address city",
    "ownerAddrState": "owner address state",
    "ownerAddrZip": "owner address ZIP",
    "ownerAddrCountry": "owner address country",
    "taxAgentID": "tax agent ID",
    "taxAgentName": "tax agent name",
    "imprvYearBuilt": "improvement year built",
    "currValYear": "current value year",
    "currValImprv": "current value improvement",
    "currValLand": "current value land",
    "currValMarket": "current value market",
    "currValAgLoss": "current value agriculture loss",
    "currValAppraised": "current value appraised",
}

key_for_search = {
    "propID": "property ID",
    "legalDescription": "legal description",
    "dbaName": "doing business as name",
    "situsBldgNum": "situs building number",
    "situsStreetPrefix": "situs street prefix",
    "situsStreetName": "situs street name",
    "situsStreetSuffix": "situs street suffix",
    "situsUnit": "situs unit",
    "situsCity": "situs city",
    "situsZip": "situs ZIP",
    "situsConcat": "situs concatenated",
    "ownerID": "owner ID",
    "ownerName": "owner name",
    "ownerNameAddtl": "owner name additional",
}

def preprocess_csv(file_path):
    df = pd.read_csv(file_path)
    df.rename(columns=key_replacements, inplace=True)
    filtered_df = df[list(key_for_search.values())]
    return df, filtered_df

def convert_to_documents(df):
    return [row.to_dict() for _, row in df.iterrows()]

def get_documents_facts(document: dict):
    facts = ""
    for key, value in document.items():
        facts += f"{key}: {value}\n"
    return facts


def create_node_representation(input_dict):
    # Extract relevant fields
    legal_description = input_dict.get("legal description", "N/A")
    owner_id = input_dict.get("owner ID", "N/A")
    owner_name = input_dict.get("owner name", "N/A")
    owner_name_additional = input_dict.get("owner name additional", "N/A")
    property_id = input_dict.get("property ID", "N/A")
    situs_concatenated = input_dict.get("situs concatenated", "N/A")
    situs_city = input_dict.get("situs city", "N/A")
    situs_zip = input_dict.get("situs ZIP", "N/A")
    dba = input_dict.get("doing business as name", "N/A")
    
    # Combine owner names if additional owner exists
    if (owner_name_additional in ["N/A", "nan"] or pd.isna(owner_name_additional)):
        owner_names = owner_name
    else:
        owner_names = f"{owner_name} and {owner_name_additional}"

    # Construct the natural language description
    value = (
        f"This property, located at {situs_concatenated}, is legally described as "
        f"'{legal_description}'. The property ID is {property_id}. "
        f"It is owned by {owner_names}, with an owner ID of {owner_id}. "
    )
    if not(dba == "N/A" or pd.isna(dba)):
        value += f"The property is also known as {dba}."

    # Construct the output dictionary
    output_node = {
        "key": f"Property Information - {situs_concatenated}",
        "value": value,
        "metadata": {
            "property_id": property_id,
            "situs_city": situs_city,
            "situs_zip": situs_zip,
            "owner_name": owner_names,
            "legal_description": legal_description
        }
    }
    return output_node


def get_nodes(input_file_path):
    df, filtered_df = preprocess_csv(input_file_path)
    documents = convert_to_documents(filtered_df)
    full_nodes = []
    owner_nodes = []
    all_nodes = []
    node_id = 0
    for document in documents:
        doc_node = create_node_representation(document)
        node_text = f"{doc_node['key']}. {doc_node['value']}"
        node_full = TextNode(text=node_text, 
                             id_=str(node_id),
                             metadata=doc_node["metadata"])
        node_owner = TextNode(text=doc_node["metadata"]["owner_name"],
                              id_=str(node_id)+"_owner",
                              metadata={
                                  "owner_name":doc_node["metadata"]["owner_name"],
                                  "legal_description":doc_node["metadata"]["legal_description"],
                              }
                              )
        node_full.relationships[NodeRelationship.CHILD] = [RelatedNodeInfo(node_id=node_owner.node_id)]
        node_owner.relationships[NodeRelationship.PARENT] = RelatedNodeInfo(node_id=node_full.node_id)
        node_id += 1        
        full_nodes.append(node_full)
        owner_nodes.append(node_owner)
        all_nodes.append(node_full)
        all_nodes.append(node_owner)

    return full_nodes, owner_nodes, all_nodes

In [7]:
property_file_path = os.path.join(data_path, "Collin_CAD_Appraisal_Data_2024_20241208_75024.csv")
# df, filtered_df = preprocess_csv(property_file_path)
full_nodes, owner_nodes, all_nodes = get_nodes(property_file_path)

/var/folders/_x/tnypr3r90l1301hrh8vy08jm0000gn/T/ipykernel_19729/2476547042.py:59: DtypeWarning: Columns (32,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [8]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
leaf_nodes = get_leaf_nodes(all_nodes)
root_nodes = get_root_nodes(all_nodes)
print(f"Number of leaf nodes: {len(leaf_nodes)}")
print(f"Number of root nodes: {len(root_nodes)}")

Number of leaf nodes: 16854
Number of root nodes: 16854


In [9]:
from llama_index.core.schema import MetadataMode
# This is what is visible to the embeddings model
print("*"*50)
print("Visible to Embeddings model")
print(full_nodes[0].get_content(metadata_mode=MetadataMode.EMBED))

# This is what is visible to the LLM
print("*"*50)
print("Visible to LLM")
print(full_nodes[0].get_content(metadata_mode=MetadataMode.LLM))

**************************************************
Visible to Embeddings model
property_id: 2005248
situs_city: PLANO
situs_zip: 75024.0
owner_name: KING DAVID H
legal_description: BRADFORD ESTATES, BLK A, LOT 1

Property Information - 4001 CAMROSE DR , PLANO, TX 75024. This property, located at 4001 CAMROSE DR , PLANO, TX 75024, is legally described as 'BRADFORD ESTATES, BLK A, LOT 1'. The property ID is 2005248. It is owned by KING DAVID H, with an owner ID of 299468.
**************************************************
Visible to LLM
property_id: 2005248
situs_city: PLANO
situs_zip: 75024.0
owner_name: KING DAVID H
legal_description: BRADFORD ESTATES, BLK A, LOT 1

Property Information - 4001 CAMROSE DR , PLANO, TX 75024. This property, located at 4001 CAMROSE DR , PLANO, TX 75024, is legally described as 'BRADFORD ESTATES, BLK A, LOT 1'. The property ID is 2005248. It is owned by KING DAVID H, with an owner ID of 299468.


In [10]:
print(full_nodes[10])
print(owner_nodes[10])

Node ID: 10
Text: Property Information - 4113 CAMROSE DR , PLANO, TX 75024. This
property, located at 4113 CAMROSE DR , PLANO, TX 75024, is legally
described as 'BRADFORD ESTATES, BLK A, LOT 11'. The property ID is
2005258. It is owned by HAYES JESSEMINE, with an owner ID of 1071053.
Node ID: 10_owner
Text: HAYES JESSEMINE


In [11]:
owner_nodes[0].relationships

{<NodeRelationship.PARENT: '4'>: RelatedNodeInfo(node_id='0', node_type=None, metadata={}, hash=None)}

In [12]:
full_nodes[0].relationships

{<NodeRelationship.CHILD: '5'>: [RelatedNodeInfo(node_id='0_owner', node_type=None, metadata={}, hash=None)]}

## Instantiate Document Store

In [13]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(all_nodes)

## Instantiate Index

In [14]:
import faiss

# dimensions of text-ada-embedding-002 1536
# dim BAAI/bge-large-en-v1.5 1024
d = 1024
faiss_index = faiss.IndexFlatL2(d)

vector_store = FaissVectorStore(faiss_index=faiss_index)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore, vector_store=vector_store)

In [15]:
# owner_nodes are defined as leaf nodes
owner_index = VectorStoreIndex(nodes=owner_nodes,
                               embed_model=embedding_model,
                               index_name="owner_index",
                               show_progress=True,
                               insert_batch_size=10000,
                               storage_context=storage_context,
                              )
print("Owner index created")

Generating embeddings:   0%|          | 0/10000 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/6854 [00:00<?, ?it/s]

Owner index created


## Define Retriever

In [16]:
from llama_index.core.retrievers import AutoMergingRetriever

base_retriever = owner_index.as_retriever(similarity_top_k=20)
auto_merge_retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=False)

In [17]:
# query_str = "What were some lessons learned from red-teaming?"
# query_str = "Can you tell me about the key concepts for safety finetuning"
query_str = (
    "What properties are owned by 'Jacob'"
)

nodes = auto_merge_retriever.retrieve(query_str)
print(f"Number of retrieved nodes: {len(nodes)}")
for node in nodes:
    print(node)
    

Number of retrieved nodes: 20
Node ID: 3909
Text: Property Information - 4408 OVERTON DR , PLANO, TX 75074. This
property, located at 4408 OVERTON DR , PLANO, TX 75074, is legally
described as 'HUDSON HEIGHTS PHASE ONE, BLK H, LOT 24'. The property
ID is 2704055. It is owned by SIDDIKI ALI, with an owner ID of
1309718.
Score:  0.657

Node ID: 3614
Text: Property Information - 1505 SWEETWATER CIR , SACHSE, TX 75048.
This property, located at 1505 SWEETWATER CIR , SACHSE, TX 75048, is
legally described as 'SACHSE WEST ESTATES, BLK A, LOT 11'. The
property ID is 1564765. It is owned by MAK JOSHUA A, with an owner ID
of 1176059.
Score:  0.656

Node ID: 11400
Text: Property Information - 4200 HEDGCOXE RD , PLANO, TX 75024. This
property, located at 4200 HEDGCOXE RD , PLANO, TX 75024, is legally
described as 'HEDGCOXE, BLK A, LOT 1R; REPLAT'. The property ID is
2527726. It is owned by SPIRITUAL ASSEMBLY OF BAHAIS, with an owner ID
of 430611. The property is also known as BAHAI FAITH PLANO CE

In [18]:
query_str = (
    "What properties are owned by 'Smith'"
)

nodes = auto_merge_retriever.retrieve(query_str)
print(f"Number of retrieved nodes: {len(nodes)}")
for node in nodes:
    print(node)

Number of retrieved nodes: 20
Node ID: 4736
Text: Property Information - 4300 HEATH CT , PLANO, TX 75024. This
property, located at 4300 HEATH CT , PLANO, TX 75024, is legally
described as 'ESTATES AT FOUNTAIN CREEK II PHASE IIB, BLK C, LOT 68'.
The property ID is 2036938. It is owned by SMITH PRESTON & ANGELA,
with an owner ID of 1269794.
Score:  0.624

Node ID: 9897
Text: Property Information - 6605 INDIAN TRL , PLANO, TX 75024. This
property, located at 6605 INDIAN TRL , PLANO, TX 75024, is legally
described as 'SHOAL CREEK PHASE III, BLK H, LOT 33'. The property ID
is 2068181. It is owned by SMITH PHILIP E & and MELANI WILSON SMITH,
with an owner ID of 1159221.
Score:  0.624

Node ID: 5900
Text: Property Information - 4049 WHITE PORCH RD , PLANO, TX 75024.
This property, located at 4049 WHITE PORCH RD , PLANO, TX 75024, is
legally described as 'HICKORY RIDGE, BLK N, LOT 7'. The property ID is
2142396. It is owned by SMITH JOSHUA WAYNE, with an owner ID of
1109212.
Score:  0.624

No

In [19]:
auto_merge_retriever

## Query Engine

In [20]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever=auto_merge_retriever,
                                              llm=generation_llm)

In [21]:
query_list = [
        ("What properties are owned by 'Smith'"),
        ("What properties are owned by 'Smeeth'"),
        ("What properties are owned by 'Smithes'"),
    ]

for query_str in query_list:
    response = query_engine.query(query_str)
    print(f"Query: {query_str}")
    print(str(response))
    print("*"*50)

Query: What properties are owned by 'Smith'
The following properties are owned by individuals with the last name 'Smith':

1. 4300 HEATH CT, PLANO, TX 75024 - Owned by SMITH PRESTON & ANGELA
2. 6605 INDIAN TRL, PLANO, TX 75024 - Owned by SMITH PHILIP E & MELANI WILSON SMITH
3. 4049 WHITE PORCH RD, PLANO, TX 75024 - Owned by SMITH JOSHUA WAYNE
4. 4552 JENKINS DR, PLANO, TX 75024 - Owned by SMITH CECELIA P
5. 9229 OLD VERANDA RD, PLANO, TX 75024 - Owned by SMITH MICHAEL G & JENQ-LIANG LIN
6. 4701 TORY HILL CT, PLANO, TX 75024 - Owned by SMITH TATIANA & ROMAN GOUBIN
7. 4009 BRIAR BLUFF TER, PLANO, TX 75024 - Owned by SMITH ROBERT J & KIRSTEN L
8. 8005 FALLMEADOW CIR, PLANO, TX 75024 - Owned by SMITH DOUGLAS
9. 7942 BISHOP RD, PLANO, TX 75024 - Owned by SMITH HENRY LEE LIVING TRUST
10. 4009 NORCROSS DR, PLANO, TX 75024 - Owned by SMITH MONSERRAT
11. 6805 MULHOUSE CT, PLANO, TX 75024 - Owned by SMITH JAMES & DEBORAH SMITH
12. 6849 COLONNADE DR, PLANO, TX 75024 - Owned by SMITH SHERRY W
13. 

In [22]:
query_list = [("List all owners with name resembling 'Jacob'"),]

for query_str in query_list:
    response = query_engine.query(query_str)
    print(f"Query: {query_str}")
    print(str(response))
    print("*"*50)

Query: List all owners with name resembling 'Jacob'
1. JACOB GEORGE CHEMPITHRA - Owner of the property located at 4008 BREWER DR, PLANO, TX 75024.
2. SAMUEL JACOB & NANCY - Owners of the property located at 4555 CAPE CHARLES DR, PLANO, TX 75024.
3. JACOB SUNNY E & JACOB ANNAMMA C - Owners of the property located at 8716 OHIO DR, PLANO, TX 75024.
4. JACOB - THANGARAJ ABRAHAM STEPHEN & JIJI JOSEPH EMMANUEL DHAS - Owners of the property located at 4308 NARBERTH DR, PLANO, TX 75024.
5. GEBHART JACOB R & NATALIE A - Owners of the property located at 4519 HELSTON DR, PLANO, TX 75024.
**************************************************


In [23]:
query_list = [("List all owners with name resembling 'Henry'"),]

for query_str in query_list:
    response = query_engine.query(query_str)
    print(f"Query: {query_str}")
    print(str(response))
    print("*"*50)

Query: List all owners with name resembling 'Henry'
1. HARRIS HENRY L - Owner ID: 875219, Property ID: 1995101, Address: 4420 ENGLISH OAK DR, PLANO, TX 75024.
2. TOW HENRY & SHARON - Owner ID: 737883, Property ID: 11997, Address: 4564 SOUTHGATE DR, PLANO, TX 75024.
3. HENRY CHRISTOPHER A CPA - Owner ID: 759556, Property ID: 2651424, Address: 6302 WINDCREST DR, PLANO, TX 75024.
**************************************************
